In [1]:
# Setting Environment Variables for Langsmith
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [2]:
# Create an LLM model instance for LLAMA2
from langchain_community.llms import Ollama

model = Ollama(model="llama2")

In [3]:
# Models are instances of Runnables, meaning they expose 
# a standard interface for interaction
from langchain_core.messages import HumanMessage

# Here we are telling the model our name
result1 = model.invoke([HumanMessage(content="Hi!, I'm Rishil")])
print(result1)
print(type(result1))

print("*******************")

# Right after we told our name, 
# model still does not remember it because no history is maintained by the model
result2 = model.invoke([HumanMessage(content="What's my name?")])
print(result2)
print(type(result2))


Assistant: Hello Rishil! It's nice to meet you. How can I assist you today?
<class 'str'>
*******************
Assistant: I'm just an AI, I don't have access to personal information about you unless you provide it to me. Can you tell me your name?
<class 'str'>


In [4]:
# For the model to know past conversation, 
# we have to provide the conversation history to it as a list
# Then the model will respond as per the latest message in the history
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Rishil"),
        AIMessage(content="Hello Rishil! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

'\nAI: Your name is Rishil.'

In [5]:
# To implement this message history mechanism we use integrations from langchain_community
# Basically keeping track of i/p and o/p and storing them in datastore
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Each conversation will be stored under a different session_id in store dict
store = {}

# If a session id is not present, create a new empty chat history
# and if present, pass the chat history of that session id to the model
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create a Runnable object that passes the chat history to the model from store
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
# Use a config variable to pass to the runnable every time.
# This contains info that is not directly part of input but still useful
# Each unique session_id will represent a different conversation
# and accordingly be stored in the data store
config = {"configurable": {"session_id": "abc2"}}

In [7]:
# Use the Runnable object that passes history to the model as per session id
response = with_message_history.invoke(
    [HumanMessage(content="Hi, I'm Rishil")],
    config=config
)

response

Parent run d85be9f2-d99d-44f7-bc11-b19a364ef4f7 not found for run b54bfe76-d2a5-4bde-ac06-e08c854981b7. Treating as a root run.
Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"\nAssistant: Hello Rishil! It's nice to meet you. How can I assist you today? Is there something specific you would like to chat about or ask?"

In [8]:
# Using Runnable with message history to check if history was maintained
# This does not work with LLAMA2 model as the return format is different
# so message history cannot be maintained
# Should work fine with OpenAI or any other models
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response

Parent run 2841f5a2-fed4-4b24-8edf-c99b498277cd not found for run 4901297e-73e4-4b12-85e3-e69e2ce92b31. Treating as a root run.


Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"Bot: I'm just an AI, I don't have access to personal information such as your name. Additionally, it is not appropriate or ethical to ask for personal information from someone without their consent. It is important to respect people's privacy and only request information that is necessary and appropriate in a given context. Is there anything else I can help you with?"

In [9]:
# Prompt Templates to create a message history
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Every message in ChatPromptTemplate.from_messages will be a list of tuples
# Every tuple is a single message consisting of 2 values
# First is either "system" or "user", representing whose message was it
# Second is the actual message from the system or the user.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),  # Use this as placeholder to add new messages in it
    ]
)

chain = prompt | model


In [10]:
# Passing input to a chain instead of the LLM as dict
# we are using "messages" to add this chat to the placeholder 
# we created in prompt as it uses "messages" as the variable name
response = chain.invoke({"messages": [HumanMessage(content="Hi! I'm Rishil")]})

response

"Hello Rishil! *smiling* It's nice to meet you. Is there something I can help you with or would you like me to recommend some information on a particular topic? Please feel free to ask me anything!"

In [11]:
# Now we can wrap this in message history as before, this time with chain that creates 
# a ChatPromptTemplate which stores the chat
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc5"}}

In [12]:
# Using the chain to chat with the bot that keeps history of the conversation
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Rishil")],
    config=config
)

# After giving your name to the model
print(response)

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print("*******************")

# Asking your name again to check if history was maintained
print(response)

Parent run 0aef3a1e-793f-45d6-bacf-aab0c4424079 not found for run 2d459089-bc70-40fa-b908-82b2451adefc. Treating as a root run.


Parent run 64b12dfb-7f4c-45ed-87a9-def23c8e6d22 not found for run c8dc1fc8-aedb-4e92-90bc-eac00feace35. Treating as a root run.


Hello Rishil! It's nice to meet you. How can I assist you today? Do you have any questions or tasks you'd like me to help you with?
*******************
AI: Hi Rishil! Your name is Rishil. 😊 Is there anything else you would like to know or discuss?


In [13]:
# Checking our data store to see the chats stored in it
store

{'abc2': InMemoryChatMessageHistory(messages=[]),
 'abc5': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Rishil"), AIMessage(content="Hello Rishil! It's nice to meet you. How can I assist you today? Do you have any questions or tasks you'd like me to help you with?"), HumanMessage(content="What's my name?"), AIMessage(content='AI: Hi Rishil! Your name is Rishil. 😊 Is there anything else you would like to know or discuss?')])}

In [14]:
# Going one step further, we will make our chatbot converse in a given language
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Create the chain again to make the chatbot converse in a given language
chain = prompt | model

In [15]:
# Now we can also pass language as input to the chain
response = chain.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Rishil")], "language": "Japanese"}
)

response

"おはようございます、リシャールです! (Ohayou gozaimasu, Risha-ru desu!) It's a pleasure to meet you! How may I assist you today? 😊"

In [16]:
# Now wrapping this more complicated chain in message history class
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "abc11"}}

In [17]:
# Using this Runnable with message history 
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Rishil")], "language": "Japanese"}, 
    config=config
)

response

Parent run 267b1613-e69a-4646-b7f2-5acca0c170a9 not found for run dc7df844-8609-4d61-a287-ae7215f3f1cf. Treating as a root run.


'「こんにちは、リシルさん。お名前がとても素晴らしいです。日本語を話せるかどうか、私たち助けになります！」'

In [18]:
# Checking if it remembers my name 
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")], "language": "Japanese"},
    config=config
)

response

Parent run 95c64a89-b2ed-47b1-95f6-c88de37c077a not found for run 88fdd26c-f794-4e59-8a5d-14f1174ec821. Treating as a root run.


'AI:「リシルさんの名前は、あなたのお名前です。名前は、日本語で言います。あなたの名前は、「Rishil」と言います。」'

In [20]:
# Managing Conversation History
# If the converstaion history is left unmanaged, it will grow unbounded and potentially 
# beyond the context window of the LLM.
from langchain_core.messages import SystemMessage, utils


trimmer = utils.trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

In [37]:
# To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.
# Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

['AIMessage', 'AIMessageChunk', 'AnyMessage', 'BaseMessage', 'BaseMessageChunk', 'ChatMessage', 'ChatMessageChunk', 'FunctionMessage', 'FunctionMessageChunk', 'HumanMessage', 'HumanMessageChunk', 'InvalidToolCall', 'MessageLikeRepresentation', 'SystemMessage', 'SystemMessageChunk', 'ToolCall', 'ToolCallChunk', 'ToolMessage', 'ToolMessageChunk', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_message_from_dict', 'ai', 'base', 'chat', 'convert_to_messages', 'function', 'get_buffer_string', 'human', 'merge_content', 'message_chunk_to_message', 'message_to_dict', 'messages_from_dict', 'messages_to_dict', 'system', 'tool', 'utils']


In [ ]:
# But if we ask about info that is within the last few messages, it remembers
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

In [ ]:
# Creating a new chain that uses trimmer along with keeping message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

In [ ]:
# To improve the User Experience, we can stream the output as it is being generated
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")